# Libraries and Installations

In [1]:
'''!pip install sagemaker-experiments
!pip install s3fs
!pip install matplotlib
!pip install seaborn
!pip install shap
!pip install smdebug
!pip install sagemaker==2.35.0
'''

  Using cached botocore-1.20.49-py2.py3-none-any.whl (7.4 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.62
    Uninstalling botocore-1.20.62:
      Successfully uninstalled botocore-1.20.62


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

boto3 1.17.57 requires botocore<1.21.0,>=1.20.57, but you'll have botocore 1.20.49 which is incompatible.
awscli 1.19.60 requires botocore==1.20.60, but you'll have botocore 1.20.49 which is incompatible.


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

awscli 1.19.60 requires botocore==1.20.60, but you'll have botocore 1.20.64 which is incompatible.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you'll have botocore 1.20.64 which is incompatible.


  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.49
    Uninstalling botocore-1.20.49:
      Successfully uninstalled botocore-1.20.49


In [3]:
from io import StringIO
import numpy as np
import os
import pandas as pd
import boto3
import time
import s3fs
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import re
#import shap
from scipy import stats
import copy

In [4]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig,CollectionConfig
from sagemaker.estimator import Estimator
from sagemaker.session import s3_input
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import Session

from smdebug.trials  import create_trial

[2021-05-04 08:55:54.869 TIGER0920:20736 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


# Configs

In [5]:
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local':{'local_code':True}}

In [6]:
now = datetime.now()

current_time = now.strftime("%Y-%m-%d--%H-%M-%S")
print("current_time:",current_time)

#sagemaker_session = sagemaker.Session()

# Change "get_execution_role()" to access role name for local deployment
role = 'arn:aws:iam::752400441523:role/Sagemaker_Access'

bucket = '1905-assignment2-sm'
prefix = 'Scikit-pre-model-Inference-Pipelines'

train_data = 's3://1905-assignment2-sm/housing/imput-datasets/train_data_without_header.csv'
test_data = 's3://1905-assignment2-sm/housing/imput-datasets/test_data_without_header.csv'

FRAMEWORK_VERSION = "0.23-1"
script_path = 'sklearn_pipeline.py'
dependency_path ='dependencies.py'

base_job_name = f"Builtin-XGB-algo-{current_time}"

output_data_prefix = f'housing/datasets/output/{base_job_name}'
data_output_path = f's3://{bucket}/{output_data_prefix}'

debug_prefix = f'housing/jobs/debug/{base_job_name}'
debug_path = f's3://{bucket}/{debug_prefix}'

experiment_name_prefix = "builtin-xgboost-track13"

current_time: 2021-05-04--08-55-55


In [7]:
print(train_data)

s3://1905-assignment2-sm/housing/imput-datasets/train_data_without_header.csv


In [8]:
role

'arn:aws:iam::752400441523:role/Sagemaker_Access'

# Batch transform

## Fit the train data

In [9]:
sklearn_preprocessor = SKLearn(
    entry_point = script_path,
    role = role,
    framework_version = FRAMEWORK_VERSION,
    train_instance_type = 'local', # "ml.m5.xlarge", #"local" ,
    #train_use_spot_instance = True,
    train_max_run = 600,
    train_max_wait = 1200,
    dependencies = [dependency_path],
    #sagemaker_session = sagemaker_session
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.


In [10]:
role

'arn:aws:iam::752400441523:role/Sagemaker_Access'

In [11]:
check = pd.read_csv(train_data)
check.head()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY,358500.0
0,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY,352100.0
1,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY,341300.0
2,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY,342200.0
3,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,NEAR BAY,269700.0
4,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,NEAR BAY,299200.0


In [12]:
train_data

's3://1905-assignment2-sm/housing/imput-datasets/train_data_without_header.csv'

In [13]:
sklearn_preprocessor.fit(
    inputs={'train':train_data},
    job_name=base_job_name
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: Builtin-XGB-algo-2021-05-04--08-55-55
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-l4icc:
    command: train
    container_name: azg0wscghn-algo-1-l4icc
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-l4icc
    stdin_open: true
    tty: true
    volumes:
    - C:\Users\SUDHAR~1.

Creating azg0wscghn-algo-1-l4icc ... 
Creating azg0wscghn-algo-1-l4icc ... done
Docker Compose is now in the Docker CLI, try `docker compose up`

Attaching to azg0wscghn-algo-1-l4icc
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:46,905 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:46,919 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:47,024 sagemaker_sklearn_container.training INFO     Invoking user training script.
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:49,330 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:49,455 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:49,600 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
azg0wscghn-algo-1-l4ic

azg0wscghn-algo-1-l4icc | saved model!
azg0wscghn-algo-1-l4icc | 2021-05-04 03:26:53,515 sagemaker-containers INFO     Reporting training SUCCESS
azg0wscghn-algo-1-l4icc exited with code 0
Aborting on container exit...
===== Job Complete =====


## Transform the training data

In [14]:
transformer = sklearn_preprocessor.transformer(
    instance_count=1,
    instance_type='local',#'ml.m5.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv',
    #output_path=data_output_path
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2021-05-04-03-26-57-251


In [ ]:
transformer.transform(
    data=train_data,
    content_type="text/csv",
    #job_name='sagemaker-scikit-learn-2021-05-03-13-50-09-847'+'-train'
)

INFO:sagemaker:Creating transform job with name: sagemaker-scikit-learn-2021-05-04-03-27-15-875
INFO:sagemaker.local.image:serving
INFO:sagemaker.local.image:creating hosting dir in C:\Users\SUDHAR~1.MUN\AppData\Local\Temp\tmpnw0atrix
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-jcw7p:
    command: serve
    container_name: fus014alpz-algo-1-jcw7p
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-jcw7p
    ports:
    - 8080:8080
    stdin_open: true
    tty: true
    volumes:
    - C:\Users\SUDHAR~1.MUN\AppData\Local\Temp\tmpf6z_tvly:/opt/ml/model
version: '2.3'

INF

Docker Compose is now in the Docker CLI, try `docker compose up`



INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 10
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 15
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 20
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 25
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 30
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 35
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 40
INFO:sagemaker.local.entities:Container still not up, got: 500
INFO:s

In [ ]:
print("Waiting for transform job:" + transformer.latest_transform_job.job_name)
transformer.wait()

In [ ]:
preprocessed_train_data = transformer.output_path

In [ ]:
preprocessed_train_data

## Transform the test data

In [ ]:
transformer.transform(
    data=test_data,
    content_type="text/csv",
    job_name=base_job_name+"-test")

print("Waiting for transform job:" + transformer.latest_transform_job.job_name)
transformer.wait()

In [ ]:
preprocessed_test_data = transformer.output_path

In [ ]:
f'{output_data_prefix}'

## Real time Prediction using endpoint

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel

timestamp_prefix = current_time

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
scikit_learn_inferencee_model.env = {"SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT":"text/csv"}
model_containers = [scikit_learn_inferencee_model]

model_name = 'inference-pipeline-' + current_time
endpoint_name = 'inference-pipeline-ep-' + current_time

#sm_model = PipelineModel(
#            name=model_name,
#            role=role,
#            models=model_containers)

predictor = scikit_learn_inferencee_model.deploy(initial_instance_count=1,
                           instance_type='local',#'ml.m5.xlarge',
                           endpoint_name=endpoint_name,
                           #data_capture_config=data_capture_config
                           )

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

payload = "-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY"

predictor = Predictor(
        endpoint_name = endpoint_name,
        sagemaker_session = sagemaker_session,
        serializer = CSVSerializer(),
        deserializer = JSONDeserializer(),
    )


In [ ]:
print(predictor.predict(data=payload))

In [ ]:
#Delete the endpoint
#sm_client = sagemaker_session.boto_session.client('sagemaker')
#sm_client.delete_endpoint(EndpointName=endpoint_name)